# Explanation of the "<<" operation

## Introduction

One of the more complex operators in the fiber tree execution model is the "<<" operator, which is used when performing a computation in which values are being inserted into (or used to update values in) a fiber. 

TLDR; For the operation ```z_i << a_i``` on fibers ```z_i``` and ```a_i```, the ```<<``` operator indicates that for every coordinate of an element in ```a_i``` one wants to update the payload of an element in ```z_i``` with that same coordinate. The expression does its work in two steps: First, it takes every coordinate in ```a_i``` and if that coordinate does not exist in ```z_i``` it inserts an element at that coordinate in the ```z_i``` fiber with a "default" payload. Second, it applies the intersection operator (```&```) on the **new** ```z_i``` and **original** ```a_i``` and returns the resulting fiber (Note that newly inserted elements in ```z_i``` always survive the intersection).

There are set of invariants maintained when the expression ```f_i = z_i << a_i``` is executed:

- coords(f_i) == coords(a_i)
- coords(z_i') == coords(a_i) + coords(z_i), where ```z_i'``` is the value of ```z_i``` after the expression is executed


If that was too terse, the following cells will illustrate the use of the operator.

But first, we include some libraries and provide some dropdown lists to select the display style and type of animation.

In [ ]:
# Run boilerplate code to set up environment

%run ../prelude.py --style=tree --animation=movie


## Creating a tensor

To start we will create a sparse input tensor ```a```, which we will use in the examples below.

Note the payload value for each coordinate in tensor ```a``` is one more than its coordinate value.

In [ ]:
a = Tensor.fromUncompressed(['I'], [0, 2, 0, 4, 5, 0, 7])

print("Tensor a")
displayTensor(a)


## Review of intersection operator (&)

Before we look at the ```<<``` operator, let's review the intersection operator. To do that we introduce a new tensor ```b```, which has contains some of the coordinates in tensor ```a``` and some that are not in ```a```. The payloads of ```b``` are 11 more than their coordinate value. 

The code below intersects the root fibers of ```a``` and ```b``` which creates a new fiber with only coordinates that existed in both ```a``` and ```b``` and payloads that are tuples whose contents are the payloads of those coordinates that survive the intersection from tensors ```a``` and ```b```, respectively. So the results are: 


- coordinate 0: Is neither in ```a``` or ```b```, so does NOT exist in the result
- coordinate 1: Is in both ```a``` and ```b```, so does exist in the result with a payload (2, 12)
- coordinate 2: Is only in ```b```, so does NOT exist in the result
- coordinate 3: Is only in ```a```, so does NOT exist in the result
- coordiante 4: Behaves like coordinate 1
- coordinate 5: Behaves like coordinate 0
- coorindate 6: Behaves like coordinate 3

So in the final figure below, we see that only coordinates 1 and 4 survive the insersection and their payloads are the tuples (2, 12) and (5, 15), where tuples are represented as red payload boxes with two values listed vertically.

In [ ]:
#
# Get the a_i fiber
#
a_i = a.getRoot()

print("Tensor a")
displayTensor(a)

#
# Create the b tensor, and extract its root fiber
#
b = Tensor.fromUncompressed(['I'], [0, 12, 13, 0, 15, 0, 0])
b_i = b.getRoot()

print("Tensor b")
displayTensor(b)

#
# Compute the intersection of a_i and b_i
#
f_i = a_i & b_i

print("Fiber f")
displayTensor(f_i)

## Creating an output for every coordinate of an input fiber

Suppose we want to create an output in a rank-1 tensor ```z``` for every non-empty coordinate in a rank-1 input tensor ```a```. To do this, we would like to populate an initially empty tensor ```z``` with the "default" value (in this case zero) for each non-empty element of ```a```. To accomplish this, we can use the fiber tree ```<<``` operator. However, since the ```<<``` operator works on **fibers** we first get the root fibers (which are both of rank "I") out of the ```a``` and ```z``` tensors, and then apply the ```<<``` operator.

Since the fiber ```z_i``` was initially empty, for every non-empty coordinate in ```a_i```, a coordinate and payload are inserted in ```z_i```.  Furthermore, since the fiber ```z_i``` is a "leaf" fiber of tensor ```z```, we see that tensor ```z``` specifically has a "default" payload value of zero inserted at those coordinates. (Foreshadow: later we will see that if the left-hand operand of ```<<``` is a non-leaf fiber, the "default" payload value will be an empty fiber instead of a scalar zero).

Finally, note that the ```a_i``` fiber is unchanged.


In [ ]:
#
# Get the a_i fiber
#
a_i = a.getRoot()

print("fiber a_i - before")
displayTensor(a_i)

#
# Create an empty z tensor and get its root fiber
#
z = Tensor(rank_ids=['I'])
z_i = z.getRoot()

print("tensor z - before")
displayTensor(z)

print("fiber z_i - before")
displayTensor(z_i)

#
# Invoke the "<<" operator, and show what happened
#
z_i << a_i

print("fiber z_i - after")
displayTensor(z_i)

print("tensor z - after")
displayTensor(z)

print("fiber a_i - after (unchanged)")
displayTensor(a_i)

## Exploring the output of the "<<" operator

In reality, the ```<<``` also produces a result fiber (like the intersection operator &). So after running the cell below you will see that the "<<" operator has created a new fiber populated with only the coordinates in ```a_i```. The payloads of that fiber are tuples of the entries at those coordinates in both ```z_i``` and ```a_i``` (essentially like the intersection of the **new** ```z_i``` and ```a_i```). In fact, the values in the tuples are actually a "reference" to the values in ```z_i```, which will be relevant in the next step.

Note: we need to recreate ```z``` since we modified it above.

In [ ]:
#
# Get the a_i fiber
#
a_i = a.getRoot()

print("fiber a_i - before")
displayTensor(a_i)

#
# Create an empty z tensor and get its root fiber
#
z = Tensor(rank_ids=["I"])
z_i = z.getRoot()

print("fiber z_i - before")
displayTensor(z_i)

#
# Get the result of the "<<" operator and show the results
#
f_i = z_i << a_i

print("fiber z_i - after")
displayTensor(z_i)

print("Fiber f")
displayTensor(f_i)

## Iterating over the result of the "<<" operator

The principal use of the ```<<``` operator is as the source of iterations that can be used in a **for** loop when an (output) operand fiber is going to be mutated. The iteration over the fiber generated by the ```<<``` operator returns elements of those fibers that consist of each coordinate in the fiber and its associated payload, which in this case will be a tuple of an payload of (the updated) ```z_i``` and a payload of ```a_i```. Note that as mentioned before, the payload of ```z_i``` is actually a reference to the values in ```z_i``` so can be used to update the elements of the ```z_i``` fiber. 

Below is an example use of the ```<<``` operator to copy the values in ```a``` to ```z```. In the code below, one can see that the **index** variables of the **for** loop are elements of the fiber created by ```z_i << a_i```. Those elements are unpacked as a tuple of the form ```(i, (z_ref, a_val))```, where ```i``` is a coordinate and ```(z_ref, a_val)``` is a payload tuple containing a reference to a payload in ```z``` and a payload value from ```a```. 

The body of the loop takes the value from the ```a``` tensor and reduces it into the ```z``` tensor with a ```+=``` operator. However, since we know that the initial values in ```z``` are zero (the "default" value) this works as an assignment or overwrite of a payload. This is handled cleanly by operator overloading in Python.  (Foreshadow: Using a plain ```=``` to overwrite a payload in ```z``` will not work. To address this we add an ```<<=``` operator to overwrite a payload)

In Einsum notation this computation would be:

$$
Z_i = A_i
$$

So, we see that in the end ```z``` is a copy of ```a```. 


In [ ]:
#
# Get the a_i fiber
#
a_i = a.getRoot()

print("Tensor a")
displayTensor(a)

#
# Create an empty z tensor and get its root fiber
#
z = Tensor(rank_ids=["I"])
z_i = z.getRoot()

print("Tensor z - before")
displayTensor(z)

#
# Iterate over the elements of the output of the "<<" operator, and
# copy the payloads in a_i into z_i
#
for i, (z_ref, a_val) in z_i << a_i:
    z_ref += a_val


print("Tensor z - after")
displayTensor(z)

# More complex cases

Below we will illustrate some more complex cases involving the ```<<``` operator.


## Insertion into non-empty left-hand operands

Note that if the left-hand operand of the ```<<``` operator has non-empty payloads for some coordinates, those payloads are preserved by the ```<<``` operator, but only those payloads whose coordiantes also exist in the right-hand operand will survive the intersection and exist in (and be able to participate in the interation over) the fiber created by the ```<<``` operator. The following example has an initialized ```z``` tensor where coordinates 2 and 4 are non-empty, but only 4 also exists in the tensor ```a```.  So after the computation the payload at coordinate 2 is simply preserved (and doesn't participate in the **for** loop), while the payload at coordinate 4 is updated (does participate in the **for** loop), and the payloads from all the other coordinates in tensor ```a``` are copied to tensor ```z``` (because they participate in the **for** loop and addition with 0 is the same as a copy).

In this example, we have added an animation of the computation, which shows the initial state of the ```z_i``` tensor and the updates of the payloads for coordinates on ```a_i```.  Note, however, the ```<<``` operator operates **eagerly**, so as soon as the **for** loop is started all of the empty elements are added to the ```z_i``` fiber and show up in the aninmation - maybe earlier than is appropriate.

The code handles the various cooridinates as follows:

- At coordinate 0: Does not exist in either ```a``` or ```z``` so does exist in the final ```z```
- At coordinate 1: Exists only in ```a```, so ```a```'s payload (2) is copied to final ```z```
- At coordinate 2: Exists only in ```z```, so original value of ```z```'s payload (22) is preserved in final ```z``` 
- At coordinate 3: Behaves like coordinate 1
- At coordinate 4: Exists in both ```a``` and ```z```, so ```a```'s payload (5) is added to the ```z```'s prior payload (33) in ```z```
- At coordinate 5: Behaves like coordinate 0
- At coordinate 6: Behaves like coordinate 1

In [ ]:
#
# Get the a_i fiber
#

a_i = a.getRoot()

print("Tensor a")
displayTensor(a)

#
# Create an non-empty z tensor and get its root fiber
#
z = Tensor.fromUncompressed(['X'], [0, 0, 22, 0, 33, 0, 0])
z_i = z.getRoot()

print("Tensor z - before")
displayTensor(z)

canvas = createCanvas(a, z)

#
# Iterate over the elements of the output of the "<<" operator, and
# copy the payloads in a_i into z_i
#
for i, (z_ref, a_val) in z_i << a_i:
    z_ref += a_val
    addFrame(canvas, (i,), (i,))

print("Tensor z - after")
displayTensor(z)

displayCanvas(canvas)

## Overwrting a payload value

Up until the previous cell, we have used ```+=``` to update a payload into the output ```z``` tensor, because either we wanted to sum values in the output or were taking advantage of the fact that addition with 0 does nothing. However, if we actually want to **overwrite** the current payload value we cannot use ```+=```. For Python-related reasons, we also cannot use ```=``` because that will not follow the payload reference. Therefore, the fibertree language has yet another operator ```<<=``` that will overwrite a payload (after following the reference).

The following code uses ```<<=``` to copy tensor ```a``` into output tensor ```z```, while preserving any values previosly in ```z``` at coordinates that do not exist in ```a```. So we note the following results in the final output tensor ```z```:

- At coordinate 0: Does not exist in either ```a``` or ```z``` so does exist in the final ```z```
- At coordinate 1: Exists only in ```a```, so ```a```'s payload (2) is copied to final ```z```
- At coordinate 2: Exists only in ```z```, so original value of ```z```'s payload (22) is preserved in final ```z``` 
- At coordinate 3: Behaves like coordinate 1
- At coordinate 4: Exists in both ```a``` and ```z```, so ```a```'s payload (5) overwrites the payload in the final ```z```
- At coordinate 5: Behaves like coordinate 0
- At coordinate 6: Behaves like coordinate 1


In [ ]:
#
# Get the a_i fiber
#

a_i = a.getRoot()

print("Tensor a")
displayTensor(a)

#
# Create an non-empty z tensor and get its root fiber
#
z = Tensor.fromUncompressed(['X'], [0, 0, 22, 0, 33, 0, 0])
z_i = z.getRoot()

print("Tensor z - before")
displayTensor(z)

canvas = createCanvas(a, z)

#
# Iterate over the elements of the output of the "<<" operator, and
# copy the payloads in a_i into z_i
#
for i, (z_ref, a_val) in z_i << a_i:
    z_ref <<= a_val
    addFrame(canvas, (i,), (i,))

print("Tensor z - after")
displayTensor(z)

displayCanvas(canvas)

## Use of the ```<<``` operator on an empty rank-2 tensor

If the left-hand operand of the ```<<``` operator is not a "leaf" fiber, i.e., its payloads are fibers, then the "default" value inserted by the ```<<``` operator is an empty fiber instead of a zero. The following example illustrates this by doing the Cartesian product of rank-1 tensors ```a``` and ```b``` and putting the results in a rank-2 tensor ```z```. 

In Einsum notatation this computation would be:

$$ Z_{i,j} = A_i \times B_j $$


In [ ]:
#
# Get the a_i fiber
#
print("Tensor a")
displayTensor(a)

#
# Create an b tensor and get its root fiber
#
b = Tensor.fromUncompressed(["J"], [10, 0, 12, 13])
b_j = b.getRoot()
print("Tensor b")
displayTensor(b)

#
# Create an empty z tensor and get its root fiber
#
z = Tensor(rank_ids=["I", "J"])
z_i = z.getRoot()

print("Tensor z - before")
displayTensor(z)

canvas = createCanvas(a, b, z)

#
# In the outer loop iterate over the elements generated by the "<<" operator,
# which creates an empty fiber for the lower rank of the z tensor
# for each coordinate in a.
#
for i, (z_j, a_val) in z_i << a_i:
    #
    # In the inner loop interate over the elements generated by the "<<" operator,
    # which creates an empty scalar for the elmenets of a fiber of lower rank of the z tensor
    # for each coordinate in b
    #
    for j, (z_ref, b_val) in z_j << b_j:
        #
        # Copy the product of the payloads from the a and b tensors into z
        # Note: we can use += because z was originally empty
        #
        z_ref += a_val*b_val
        addFrame(canvas, (i,), (j,), (i,j))


print("Tensor z - after")
displayTensor(z)

displayCanvas(canvas)

## Use of the "<<" operator on a non-empty rank-2 tensor

If the left-hand operand of the ```<<``` operator is not a "leaf" fiber and is not empty then it is preserved by the "<<" operation. The following example illustrates this by doing the Cartesian product of rank-1 tensors ```a``` and ```b``` and putting the results in a rank-2 tensor ```z``` that already has some payload values.

Note how pre-existing values are either copied to the final ```z``` tensor or accumulated with the product of values from the ```a``` and ```b``` tensors.

In [ ]:
#
# Get the a_i fiber
#
print("Tensor a")
displayTensor(a)

#
# Create an b tensor and get its root fiber
#
b = Tensor.fromUncompressed(["J"], [10, 0, 12, 13])
b_j = b.getRoot()
print("Tensor b")
displayTensor(b)

#
# Create a non-empty z tensor and get its root fiber
#
z_data = [[0, 0, 1, 2],
          [0, 4, 0, 5],
          [0, 0, 0, 0],
          [0, 0, 0, 7],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
         ]

z = Tensor.fromUncompressed(["I", "J"], z_data)
z_i = z.getRoot()

print("Tensor z - before")
displayTensor(z)

canvas = createCanvas(a, b, z)

#
# In the outer loop iterate over the elements generated by the "<<" operator,
# which creates an empty fiber for the lower rank of the z tensor
# for each coordinate in a.
#
for i, (z_j, a_val) in z_i << a_i:
    #
    # In the inner loop interate over the elements generated by the "<<" operator,
    # which creates an empty scalar for the elmenets of the z tensor
    # for each coordinate in b
    #
    for j, (z_ref, b_val) in z_j << b_j:
        #
        # Copy the product of the payloads from the a and b tensors into z
        #
        z_ref += a_val*b_val
        addFrame(canvas, (i,), (j,), (i,j))

print("Tensor z - after")
displayTensor(z)

displayCanvas(canvas)

## Element-wise multiply

The right-hand operand of the ```<<``` operator can be an expression itself, as in the following example where the result of an intersection is the right-hand operand. In this case, we can see that the elements of the fiber generated by the ```<<``` operator unpack into a more complex expression ```(i, (z_ref, (a_val, b_val))```, where ```i``` is the coordinate, ```z_ref``` is a reference to a payload in the ```z``` tensor and ```(a_val, b_val)``` is a payload from a element of the fiber created by the intersection of ```a_i``` and ```b_i```.

The Einsum notation for this computation is:

$$ Z_i = A_i \times B_i $$


In [ ]:
#
# Get the a_i fiber
#
print("Fiber a_i")
displayTensor(a_i)

#
# Get the b_i fiber
#
print("Fiber b_i")
displayTensor(b_i)

#
# Create an empty z tensor and get its root fiber
#
z = Tensor(rank_ids=['I'])
z_i = z.getRoot()

print("Fiber z_i - before")
displayTensor(z_i)

#
# Iterate over the elements of the output of the "<<" operator, and
#
for i, (z_ref, (a_val, b_val)) in z_i << (a_i & b_i):
    #
    # Assign to z the product of the surviving elements of the intersection
    #
    z_ref += a_val * b_val
    
print("Fiber z_i - before")
displayTensor(z_i)


## Insert into multiple empty fibers

We can use the ```<<``` operator multiple times in a single expression. For example, the expression ```z_i << (y_i << a)``` indicates the mutation of multiple fibers with the same coordinates. The code below does a copy of a tensor ```a``` into the tensors ```y``` and ```z```.

The Einsum notation for this computation is:

$$ Y_i, Z_i = A_i $$

Note use of parenthesis to control precidence of applicaton of the ```<<``` operations.

In [ ]:
#
# Get the a_i fiber
#
a_i = a.getRoot()

print("Tensor a")
displayTensor(a)

#
# Create an empty y tensor and get its root fiber
#
y = Tensor(rank_ids=["I"])
y_i = y.getRoot()

print("Tensor y - before")
displayTensor(y)

#
# Create an empty y tensor and get its root fiber
#
z = Tensor(rank_ids=["I"])
z_i = z.getRoot()

print("Tensor z - before")
displayTensor(z)

canvas = createCanvas(a, y, z)

#
# Interate over the dual mutation of z and y
# 
for i, (z_ref, (y_ref, a_val)) in z_i << (y_i << a_i):
    y_ref += a_val
    z_ref += a_val
    addFrame(canvas, (i,), (i,), (i,))


print("Tensor y - after")
displayTensor(y)

print("Tensor z - after")
displayTensor(z)

displayCanvas(canvas)

## Insert into multiple non-empty fibers

As before the left-hand operand of the ```<<``` need not be empty. Below we see an addition of a tensor ```a``` into two non-empty tensors.

In [ ]:
#
# Get the a_i fiber
#
a_i = a.getRoot()

print("Tensor a")
displayTensor(a)

#
# Create an empty y tensor and get its root fiber
#
y = Tensor.fromUncompressed(['X'], [0, 11, 22, 0, 0, 44, 0])
y_i = y.getRoot()

print("Tensor y - before")
displayTensor(y)

#
# Create an empty z tensor and get its root fiber
#
z = Tensor.fromUncompressed(['X'], [0, 0, 22, 0, 33, 0, 0])
z_i = z.getRoot()

print("Tensor z - before")
displayTensor(z)

canvas = createCanvas(a, y, z)

#
# Interate over the dual mutation of z and y
# 
for i, (z_ref, (y_ref, a_val)) in z_i << (y_i << a_i):
    y_ref += a_val
    z_ref += a_val
    addFrame(canvas, (i,), (i,), (i,))

print("Tensor y - after")
displayTensor(y)

print("Tensor z - after")
displayTensor(z)

displayCanvas(canvas)

## Insert into multiple non-empty fibers

As before the left-hand operand of the ```<<``` need not be empty. Below we see an addition of a tensor ```a``` into two non-empty tensors.

In [ ]:
a_i = a.getRoot()

print("Tensor a")
displayTensor(a)

y = Tensor.fromUncompressed(['X'], [0, 11, 22, 0, 0, 44, 0])
y_i = y.getRoot()

print("Tensor y - before")
displayTensor(y)

z = Tensor.fromUncompressed(['X'], [0, 0, 22, 0, 33, 0, 0])
z_i = z.getRoot()

print("Tensor z - before")
displayTensor(z)

canvas = createCanvas(a, y, z)

for i, (z_ref, (y_ref, a_val)) in z_i << (y_i << a_i):
    y_ref <<= a_val
    z_ref <<= a_val
    addFrame(canvas, (i,), (i,), (i,))

print("Tensor y - after")
displayTensor(y)

print("Tensor z - after")
displayTensor(z)

displayCanvas(canvas)

## Testing area

For running alternative algorithms